
# Populate chapters of Youtube video


In [1]:
youtube_video_id="PJhl54FrGyw"
language = "ko"

# Officially no way to get chapter automatically, 
# so copy and pagese the time stamp and chapter in description of Youtube video. 

chapter_part_in_description = """
00:00 시작
08:08 정책, 제도, 사람 중에서 가장 중요한 것은?
15:56 사람에 대한 평가기준이 없는 사회에서 직무담당자(Träger)의 책임
19:06 이 채널의 앞선 강의를 듣지 않으면 연속되는 강의를 이해하기 쉽지 않다
29:04 인물에 대한 잘못된 평가기준: 신언서판(身言書判)
38:26 인재평가의 프레임워크
43:26 이낙연 현상이란 무엇인가?
1:01:44 이낙연의 공직생활
1:07:03 김수영 시인의 시 〈어느 날 고궁을 나오면서〉 (1965년 작품)
1:10:23 민주진영의 왜 분노하는가? 〈마지막 지푸라기〉
1:24:22 문재인 정부의 문재인, 이낙연, 노영민 등 3인의 태도
1:40:36 민주시민이 이낙연 현상 때문에 민주당에 등을 돌렸다
1:42:26 이재명, 추미애, 이낙연, 윤석열의 AMP지수
1:49:01 정리
"""



In [2]:
# Officially no way to get chapter automatically, 
# so we need to parse the text in description and set up the dictionary 
# [ (time_in_sec, chapter_title) ]
import re 
pattern = r'(\d+(:\d+){1,2})\s(.+)'
matches = re.findall(pattern, chapter_part_in_description)

def time_to_seconds(time):
    parts = time.split(':')
    seconds = int(parts[-1])
    minutes = int(parts[-2]) if len(parts) > 1 else 0
    hours = int(parts[-3]) if len(parts) > 2 else 0
    return hours * 3600 + minutes * 60 + seconds

chapters = [(time_to_seconds(time), title.strip()) for time, _, title in matches]


# Build up note with chapter and script under each chapter 

In [7]:
import os
from collections import deque
from youtube_transcript_api import YouTubeTranscriptApi

# Populate the script of YouTube video
data = YouTubeTranscriptApi.get_transcripts([youtube_video_id], languages=[language])
script_data = deque( data[0][youtube_video_id] )


# Put the script under each chatpter
# [ 
#   { 
#    "title": current_title,
#    "script": script_in_chapter
#    } ....
#  ]

script_by_chapter = []

script_in_chapter = ""
for i in range( len(chapters) ):
    current_time_in_sec, current_title = chapters[i]
    next_time_in_sec, next_title = chapters[i + 1] if i + 1 < len(chapters) else (None, None)


    s = script_data.popleft()
    end_time_of_script_in_sec = int( s['start'] + s['duration'] )

    if next_time_in_sec is not None:
        
        while( end_time_of_script_in_sec < next_time_in_sec ):
            script_in_chapter += s['text']
            script_in_chapter += " "
            s = script_data.popleft()
            end_time_of_script_in_sec = int( s['start'] + s['duration'] )

        chapter_data = { 
                        "title": current_title,
                        "script": script_in_chapter
                        }
        
        script_by_chapter.append(chapter_data)
        script_in_chapter = ""        

    else:
        script_in_chapter = ""

        while( len(script_data) > 0 ):
            script_in_chapter += s['text']
            script_in_chapter += " "
            s = script_data.popleft()
            end_time_of_script_in_sec = int( s['start'] + s['duration'] )

        chapter_data = { 
                        "title": current_title,
                        "script": script_in_chapter
                        }
        
        script_by_chapter.append(chapter_data)



In [4]:

# Temporary save data into file 
import os 
import json 

with open( "temp_script_by_chapter.json", "w") as file:
    file.write( json.dumps(script_by_chapter, indent=2, ensure_ascii=False) )


In [5]:
# Summarize 